## Mesh to samples

In [1]:
! pwd

/sfs/weka/scratch/rhm4nj/cral/cral-ginn/ginn/notebooks


In [4]:
import trimesh
import k3d
import numpy as np

mydir = "/scratch/rhm4nj/cral/cral-ginn/ginn"

In [5]:
file_path = f"{mydir}/GINN/simJEB/orig/411_for_envelope.obj"
mesh_jeb = trimesh.load(file_path)

# Plotting using k3d
plot = k3d.plot(name='SDF Points')
plot += k3d.mesh(mesh_jeb.vertices, mesh_jeb.faces, color=0xff0000)
plot.display()

/scratch/rhm4nj/.conda/new-ginn-env/lib/python3.13/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
/scratch/rhm4nj/.conda/new-ginn-env/lib/python3.13/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "int64" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(


Output()

In [6]:
file_path = f"{mydir}/GINN/simJEB/orig/interfaces.stl"
mesh_jeb = trimesh.load(file_path)

# Plotting using k3d
plot = k3d.plot(name='SDF Points')
plot += k3d.mesh(mesh_jeb.vertices, mesh_jeb.faces, color=0xff0000)
plot.display()

Output()

In [8]:
!pip install rtree

from math import dist
import time
import trimesh
import numpy as np
import k3d
from matplotlib import cm

file_path = f"{mydir}/GINN/simJEB/orig/411_for_envelope.obj"
mesh_jeb = trimesh.load(file_path)

# load interface mesh
file_path = f"{mydir}/GINN/simJEB/orig/interfaces.stl"
mesh_if = trimesh.load(file_path)

# sample from surface of mesh
n_iter = 2
n_pts_in_and_out = 3 * 10 ** 3  # ca. 1/3 of the points will be inside the mesh, 1/3 close to the envelope, 1/3 outside
n_pts_on_envelope = 10 ** 3
n_pts_interface = 10 ** 3
outside_env_size = 10
eps_remove_pts_on_env = 0.1  # remove points on the envelope close to the interface
n_pts_around_interface_outside_env = 10 ** 3
dist_range_pts_around_if = [-15, 15]
outside_env_size_for_pts_around_if = 15

assert n_pts_around_interface_outside_env % n_pts_interface == 0, 'n_pts_around_interface_outside_env must be a multiple of n_pts_interface'

#Plotting
point_size = 0.1
# sample_points, face_idx = mesh.sample(n_pts_in_and_out, return_index=True)
bbox_min, bbox_max = mesh_jeb.bounds
print('Mesh bounds min:', bbox_min)
print('Mesh bounds max:', bbox_max)
print('Mesh centroid:', mesh_jeb.centroid)
center_for_translation = (bbox_max + bbox_min) / 2
print('Translation vector:', center_for_translation)
scale_factor = max(bbox_max - bbox_min) / 2
print('Scale factor:', scale_factor)
bounds = np.stack([bbox_min, bbox_max])
print('Bounds:', bounds)

pts_far_outside_list = []
pts_inside_list = []
pts_on_env_list = []
pts_outside_list = []
if_points_list = []
if_normals_list = []
pts_around_interface_outside_env_list = []
for iter in range(n_iter):
    t_start = time.time()
    bounds_points = np.random.uniform(low=bbox_min - outside_env_size, high=bbox_max + outside_env_size, size=(n_pts_in_and_out, 3))
    print(f'Time after sampling points: {time.time() - t_start:.1f} seconds')
    
    # Get the signed distance for each point
    # According to the docs, the sign of the distance is negative if the point is outside the mesh
    # Revert the sign to adjust to our convention
    # https://github.com/mikedh/trimesh/blob/main/trimesh/proximity.py
    sdf_bounds_pts = (-1) * trimesh.proximity.signed_distance(mesh_jeb, bounds_points)
    print(f'Time after computing SDF: {time.time() - t_start:.1f} seconds')

    mask_far_outside = sdf_bounds_pts > outside_env_size
    pts_far_outside = bounds_points[mask_far_outside]
    print('Number of points far outside:', len(pts_far_outside))

    mask_within_close_to_env = sdf_bounds_pts < outside_env_size
    pts_within_close_to_env = bounds_points[mask_within_close_to_env]
    sdf_bounds_pts = sdf_bounds_pts[mask_within_close_to_env]
    del bounds_points  # free memory

    ## outside points
    idcs_outside = np.where(sdf_bounds_pts > 0)[0]
    pts_outside = pts_within_close_to_env[idcs_outside]
    sdf_outside = sdf_bounds_pts[idcs_outside]
    print('Number of points outside:', len(pts_outside))

    ## inside points
    idcs_inside = np.where(sdf_bounds_pts < 0)[0]
    pts_inside = pts_within_close_to_env[idcs_inside]
    sdf_inside = sdf_bounds_pts[idcs_inside]
    print('Number of points inside:', len(pts_inside))
    print(f'Time after computing inside/outside points: {time.time() - t_start:.1f} seconds')

    # remove points on the envelope close to the interface
    pts_on_env = mesh_jeb.sample(n_pts_on_envelope)
    print(f'Time after sampling points on envelope: {time.time() - t_start:.1f} seconds')
    sdf_env_to_interface = trimesh.proximity.signed_distance(mesh_if, pts_on_env)
    print(f'Time after computing SDF from envelope to interface: {time.time() - t_start:.1f} seconds')
    idcs = np.where(np.abs(sdf_env_to_interface) > eps_remove_pts_on_env)[0]
    pts_on_env = pts_on_env[idcs]
    print(f'Time after removing points on envelope close to interface: {time.time() - t_start:.1f} seconds')

    ## Sample points and determine the normals from the corresponding faces
    ## From https://github.com/mikedh/trimesh/issues/1285#issuecomment-880854466
    if_points, face_idx = mesh_if.sample(n_pts_interface, return_index=True)
    if_normals = mesh_if.face_normals[face_idx]
    print(f'Time after sampling interface points: {time.time() - t_start:.1f} seconds')
    
    # points around the interface in the envelope
    pts_around_interface_outside_env = np.repeat(if_points, n_pts_around_interface_outside_env // n_pts_interface, axis=0) + np.random.uniform(low=dist_range_pts_around_if[0], high=dist_range_pts_around_if[1], size=(n_pts_around_interface_outside_env, 3))
    sdf_around_interface_outside_env = (-1) * trimesh.proximity.signed_distance(mesh_jeb, pts_around_interface_outside_env)
    pts_around_interface_outside_env = pts_around_interface_outside_env[np.where((sdf_around_interface_outside_env > 0) & (sdf_around_interface_outside_env < outside_env_size_for_pts_around_if))[0]]
    sdf_close_to_interface = (-1) * trimesh.proximity.signed_distance(mesh_if, pts_around_interface_outside_env)
    pts_around_interface_outside_env = pts_around_interface_outside_env[np.where(np.abs(sdf_close_to_interface) > eps_remove_pts_on_env)[0]]
    print(f'Time after computing points around interface outside envelope: {time.time() - t_start:.1f} seconds')
    
    pts_far_outside_list.append(pts_far_outside)
    pts_inside_list.append(pts_inside)
    pts_on_env_list.append(pts_on_env)
    pts_outside_list.append(pts_outside)
    if_points_list.append(if_points)
    if_normals_list.append(if_normals)
    pts_around_interface_outside_env_list.append(pts_around_interface_outside_env)
    
    print(f'Iteration {iter} took {time.time() - t_start:.1f} seconds')

pts_far_outside = np.concatenate(pts_far_outside_list)
pts_inside = np.concatenate(pts_inside_list)
pts_on_env = np.concatenate(pts_on_env_list)
pts_outside = np.concatenate(pts_outside_list)
if_points = np.concatenate(if_points_list)
if_normals = np.concatenate(if_normals_list)
pts_around_interface_outside_env = np.concatenate(pts_around_interface_outside_env_list)

## Plotting
# Normalize the signed distances to [0, 1] for colormap mapping
normalized_distances = (sdf_outside - sdf_outside.min()) / (sdf_outside.max() - sdf_outside.min())
colors = cm.viridis(normalized_distances)  # Using the viridis colormap
colors_hex = [int('0x{:02x}{:02x}{:02x}'.format(int(r*255), int(g*255), int(b*255)), 16) for r, g, b, a in colors]

## Save points to file
## transpose to match the convention of the other code
# bounds = bounds.T
# print('saving bounds:', bounds)
# np.save('simJEB/bounds.npy', bounds)
# print('saving pts_far_outside:', pts_far_outside.shape)
# np.save('simJEB/pts_far_outside.npy', pts_far_outside)
# print('saving pts_outside:', pts_outside.shape)
# np.save('simJEB/pts_outside.npy', pts_outside)
# print('saving pts_inside:', pts_inside.shape)
# np.save('simJEB/pts_inside.npy', pts_inside)
# print('saving pts_on_env:', pts_on_env.shape)
# np.save('simJEB/pts_on_env.npy', pts_on_env)
# np.save('simJEB/center_for_translation.npy', center_for_translation)
# np.save('simJEB/scale_factor.npy', scale_factor)


# save the points and normals to a file
# print('saving if_points:', if_points.shape)
# np.save("simJEB/interface_points.npy", if_points)
# print('saving if_normals:', if_normals.shape)
# np.save("simJEB/interface_normals.npy", if_normals)

# print('saving pts_around_interface_outside_env:', pts_around_interface_outside_env.shape)
# np.save("simJEB/pts_around_interface_outside_env.npy", pts_around_interface_outside_env)

  Using cached Rtree-1.3.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.1 kB)
Using cached Rtree-1.3.0-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (543 kB)
Mesh bounds min: [-3.92288017e+01 -1.63356995e+02 -1.18000000e-12]
Mesh bounds max: [67.32998657 16.81476021 62.50405884]
Mesh centroid: [ 16.91079892 -71.23308014  19.42698404]
Translation vector: [ 14.05059242 -73.27111721  31.25202942]
Scale factor: 90.08587741851807
Bounds: [[-3.92288017e+01 -1.63356995e+02 -1.18000000e-12]
 [ 6.73299866e+01  1.68147602e+01  6.25040588e+01]]
Time after sampling points: 0.0 seconds
Time after computing SDF: 98.7 seconds
Number of points far outside: 1427
Number of points outside: 917
Number of points inside: 656
Time after computing inside/outside points: 98.7 seconds
Time after sampling points on envelope: 98.7 seconds
Time after computing SDF from envelope to interface: 98.9 seconds
Time after removing points on envelope close to interface: 98.9 seconds
Time a

In [ ]:
# # ## normalize the points
# bounds = (bounds - center_for_translation) / scale_factor
# pts_far_outside = (pts_far_outside - center_for_translation) / scale_factor
# pts_outside = (pts_outside - center_for_translation) / scale_factor
# pts_inside = (pts_inside - center_for_translation) / scale_factor
# pts_on_env = (pts_on_env - center_for_translation) / scale_factor
# if_points = (if_points - center_for_translation) / scale_factor
# pts_around_interface_outside_env = (pts_around_interface_outside_env - center_for_translation) / scale_factor

In [ ]:
# Plotting using k3d
plot = k3d.plot(name='SDF Points')

# envelope
plot += k3d.points(positions=pts_far_outside, point_size=point_size, color=0x000000, name='far outside') # black
plot += k3d.points(positions=pts_outside, point_size=point_size, color=0xff0000, name='outside') # red
plot += k3d.points(positions=pts_on_env, point_size=point_size, color=0x0000ff, name='on envelope') # blue
plot += k3d.points(pts_around_interface_outside_env, point_size=point_size, color=0xff00ff, name='around interface outside envelope') # magenta

plot.display()

/home/rhm4nj/.local/lib/python3.11/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(


Output()

In [ ]:
# Plotting using k3d
plot = k3d.plot(name='SDF Points')

# domain
plot += k3d.points(positions=pts_inside, point_size=point_size, color=0x00ff00, name='inside')

plot.display()

Output()

In [ ]:
# Plotting using k3d
plot = k3d.plot(name='SDF Points')

# interface
plot += k3d.points(if_points, point_size=point_size, color=0x888888, name='interface points')
plot += k3d.vectors(if_points, if_normals, color=0x444444, name='interface normals')

plot.display()

Output()

In [ ]:
# check shapes of saved files

print('bounds:', np.load('simJEB/bounds.npy').shape)
print('pts_far_outside:', np.load('simJEB/pts_far_outside.npy').shape)
print('pts_outside:', np.load('simJEB/pts_outside.npy').shape)
print('pts_inside:', np.load('simJEB/pts_inside.npy').shape)
print('pts_on_env:', np.load('simJEB/pts_on_env.npy').shape)
print('center_for_translation:', np.load('simJEB/center_for_translation.npy').shape)
print('scale_factor:', np.load('simJEB/scale_factor.npy').shape)
print('if_points:', np.load('simJEB/interface_points.npy').shaape)
print('if_normals:', np.load('simJEB/interface_normals.npy').shape)
print('pts_around_interface_outside_env:', np.load('simJEB/pts_around_interface_outside_env_5mm.npy').shape)

FileNotFoundError: [Errno 2] No such file or directory: 'simJEB/bounds.npy'